In [1]:
# --- 1. 필수 라이브러리 설치 ---
!pip install -q -U transformers peft accelerate trl datasets huggingface_hub fastapi "uvicorn[standard]" pyngrok bitsandbytes "pandas==2.2.2"


In [2]:
# --- 2. 파일 업로드 및 로그인 ---
from google.colab import files
from huggingface_hub import login
import os

print("--- 📂 파일들을 업로드합니다 ---")
files_to_upload = ['gemma-fashion-dpo-final-v3.zip', 'styles.csv']
for filename in files_to_upload:
    if not os.path.exists(filename):
        print(f"\n'{filename}'을 업로드해주세요.")
        files.upload()
    else:
        print(f"'{filename}'이(가) 이미 존재합니다.")


--- 📂 파일들을 업로드합니다 ---
'gemma-fashion-dpo-final-v3.zip'이(가) 이미 존재합니다.
'styles.csv'이(가) 이미 존재합니다.


In [3]:
# --- 3. 모델 폴더 압축 해제 및 '자동 경로 탐지' ---
print("\n모델 폴더 압축을 해제합니다...")
!unzip -q -o gemma-fashion-dpo-final-v3.zip -d /content/

ADAPTER_PATH_FOUND = ""
for root, dirs, files in os.walk('/content'):
    if 'adapter_config.json' in files:
        ADAPTER_PATH_FOUND = root
        break

if not ADAPTER_PATH_FOUND:
    raise FileNotFoundError("압축 해제 후 'adapter_config.json'을 찾을 수 없습니다. Zip 파일을 확인해주세요.")

print(f"✅ 'adapter_config.json'의 정확한 경로를 찾았습니다: {ADAPTER_PATH_FOUND}")
# ❗ 찾은 경로를 환경 변수에 저장합니다.
os.environ['ADAPTER_PATH'] = ADAPTER_PATH_FOUND



모델 폴더 압축을 해제합니다...
✅ 'adapter_config.json'의 정확한 경로를 찾았습니다: /content/content/gemma-fashion-dpo-final-v3


In [4]:
#@title 3. API 서버 코드 작성 (serve_final.py) - 최종 하이브리드 스코어링 버전

%%writefile serve_final.py

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import pandas as pd
from itertools import product
import time
import random
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List, Dict, Any
import uvicorn
import logging

# --- 기본 설정 ---
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
app = FastAPI()

# --- 전역 변수 ---
model, tokenizer, styles_info, positive_token_id, negative_token_id = None, None, None, None, None
device = "cuda" if torch.cuda.is_available() else "cpu"
outerwear_types = ['Jackets', 'Blazers', 'Waistcoat', 'Coat', 'Shrug', 'Cardigan', 'Nehru Jackets', 'Rain Jacket', 'Sweaters']
full_body_types = ['Dresses', 'Jumpsuit']

# --- 서버 시작 시 모델 로딩 ---
@app.on_event("startup")
def load_resources():
    global model, tokenizer, styles_info, positive_token_id, negative_token_id
    logger.info("✅ 최종 DPO 모델 로딩을 시작합니다...")
    adapter_path = "/content/content/gemma-fashion-dpo-final-v3"
    quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
    base_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", quantization_config=quantization_config, device_map="auto")
    model = PeftModel.from_pretrained(base_model, adapter_path)
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
    tokenizer.pad_token = tokenizer.eos_token
    positive_token_id = tokenizer.encode("긍정적", add_special_tokens=False)[0]
    negative_token_id = tokenizer.encode("부정적", add_special_tokens=False)[0]
    logger.info("✅ 모델 및 토크나이저 로딩 완료!")
    styles_df = pd.read_csv("styles.csv", on_bad_lines='skip', dtype={'id': int}).set_index('id')
    styles_info = styles_df.to_dict('index')
    logger.info("✅ styles.csv 로드 완료!")

# --- 헬퍼 함수 ---
def get_item_description(item_id):
    if item_id not in styles_info: return ""
    info = styles_info[item_id]; season = info.get('season', 'All')
    return f"{info.get('baseColour', '')} {info.get('articleType', '')} ({season})".strip()

# --- ★ NEW: 규칙 기반 '상식 점수' 계산 함수 ---
def calculate_rule_score(outfit_ids: List[int], context: Dict[str, Any]) -> float:
    """상황과 옷의 용도(usage), 아우터 착용 여부 등을 기준으로 점수를 매기는 함수"""
    score = 0.0
    items_info = [styles_info.get(id) for id in outfit_ids if id in styles_info]
    event = context['event']
    temp = context['temperature']

    # 아우터 가중치: 18°C 이하일 때 아우터가 포함된 조합에 강력한 보너스 점수 부여
    has_outer = any(info and info.get('articleType') in outerwear_types for info in items_info)
    if temp <= 18 and has_outer:
        score += 1.0 # AI 점수 스케일(0~1)에 맞춰 큰 가중치 부여
    elif temp > 22 and has_outer:
        score -= 2.0 # 더울 때 아우터는 큰 감점

    # 격식 가중치: 격식있는 자리에 모든 옷의 usage가 Formal/Smart Casual이면 보너스 점수
    if event in ["Business Meeting", "Office Meeting", "Formal Dinner", "Job Interview"]:
        if all(info and info.get('usage') in ['Formal', 'Smart Casual'] for info in items_info):
            score += 1.0
        else: # 격식에 맞지 않는 옷이 하나라도 있으면 감점
            score -= 1.0

    return score

class OutfitRequest(BaseModel):
    closet: List[int]; event: str; temperature: float; condition: str; gender: str

@app.post("/recommend_outfit")
def recommend_outfit(req: OutfitRequest):
    start_time = time.time()
    # (이전과 동일한 필터링 및 분류, 2단계 평가 로직)
    # ... (생략) ...
    temp = req.temperature; event = req.event
    if temp <= 10: suitable_seasons = ['Winter', 'Fall']
    else: suitable_seasons = ['Summer', 'Spring', 'All Season']
    target_usage = ['Casual', 'Smart Casual']
    if event in ["Business Meeting", "Formal Dinner", "Office Meeting"]: target_usage = ['Formal', 'Smart Casual']
    elif event in ["Gym Workout", "Hiking", "Sports"]: target_usage = ['Sports', 'Active']
    def filter_and_classify_closet(ids_to_filter, strict_usage_check=True):
        items_classified = { 'Topwear': [], 'Bottomwear': [], 'Outerwear': [], 'Footwear': [], 'Full Body': [] }
        for item_id in ids_to_filter:
            if item_id in styles_info:
                info = styles_info[item_id]
                is_season_ok = info.get('season') in suitable_seasons or info.get('season') == 'All Season'
                is_usage_ok = not strict_usage_check or info.get('usage') in target_usage
                if is_season_ok and is_usage_ok:
                    sub_category = info.get('subCategory', ''); article_type = info.get('articleType', ''); master_category = info.get('masterCategory', '')
                    if article_type in full_body_types: items_classified['Full Body'].append(item_id)
                    elif article_type in outerwear_types: items_classified['Outerwear'].append(item_id)
                    elif sub_category == 'Topwear': items_classified['Topwear'].append(item_id)
                    elif sub_category == 'Bottomwear': items_classified['Bottomwear'].append(item_id)
                    elif master_category == 'Footwear': items_classified['Footwear'].append(item_id)
        return items_classified
    closet_items = filter_and_classify_closet(req.closet, strict_usage_check=True)
    if not (closet_items['Topwear'] and closet_items['Bottomwear'] and closet_items['Footwear']) and not (closet_items['Full Body'] and closet_items['Footwear']):
        closet_items = filter_and_classify_closet(req.closet, strict_usage_check=False)
    all_filtered_items = list(set(item for part_items in closet_items.values() for item in part_items))
    if len(all_filtered_items) > 40: all_filtered_items = random.sample(all_filtered_items, 40)
    if not all_filtered_items: return {"error": "현재 상황에 맞는 아이템이 옷장에 없습니다."}
    item_prompts = [f"상황: {req.gender}, {req.temperature}°C, {req.condition}, {req.event}\n옷: {get_item_description(item_id)}\n결과:" for item_id in all_filtered_items]
    inputs = tokenizer(item_prompts, return_tensors="pt", padding=True).to(device)
    with torch.no_grad(): outputs = model(**inputs)
    sequence_lengths = inputs['attention_mask'].sum(dim=1) - 1
    last_token_logits = outputs.logits[torch.arange(len(all_filtered_items)), sequence_lengths]
    logits_for_scoring = last_token_logits[:, [negative_token_id, positive_token_id]]
    probs = torch.softmax(logits_for_scoring, dim=-1)
    item_scores = probs[:, 1].tolist()
    scored_items = {all_filtered_items[i]: item_scores[i] for i in range(len(all_filtered_items))}
    TOP_K_ITEMS_PER_PART = 7
    top_items_by_part = { part: sorted(items, key=lambda x: scored_items.get(x, 0), reverse=True)[:TOP_K_ITEMS_PER_PART] for part, items in closet_items.items() }
    base_combinations = list(product(top_items_by_part.get('Topwear', []), top_items_by_part.get('Bottomwear', []), top_items_by_part.get('Footwear', [])))
    outer_combinations = list(product(top_items_by_part.get('Topwear', []), top_items_by_part.get('Bottomwear', []), top_items_by_part.get('Outerwear', []), top_items_by_part.get('Footwear', [])))
    full_body_combinations = list(product(top_items_by_part.get('Full Body', []), top_items_by_part.get('Footwear', [])))
    all_combinations = [c for c in base_combinations + outer_combinations + full_body_combinations if c]
    if not all_combinations: return {"error": "상위 아이템들로 유효한 조합을 만들 수 없습니다."}

    prompts = [ f"상황: {req.gender}, {req.temperature}°C, {req.condition}, {req.event}\n옷: {', '.join([get_item_description(id) for id in combo])}\n결과:" for combo in all_combinations ]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True).to(device)
    with torch.no_grad(): outputs = model(**inputs)
    sequence_lengths = inputs['attention_mask'].sum(dim=1) - 1
    last_token_logits = outputs.logits[torch.arange(len(all_combinations)), sequence_lengths]
    logits_for_scoring = last_token_logits[:, [negative_token_id, positive_token_id]]
    probs = torch.softmax(logits_for_scoring, dim=-1)
    ai_scores = probs[:, 1].tolist()

    # --- ★ NEW: 하이브리드 최종 점수 계산 ---
    final_scored_combinations = []
    for i, combo in enumerate(all_combinations):
        combo_ids = [id for id in combo if id is not None]
        rule_score = calculate_rule_score(combo_ids, req.dict())
        final_score = ai_scores[i] + rule_score
        final_scored_combinations.append((combo, final_score))

    sorted_combinations = sorted(final_scored_combinations, key=lambda x: x[1], reverse=True)

    best_combo, best_score = sorted_combinations[0]
    best_combo_ids = [int(id_val) for id_val in best_combo if id_val is not None]
    outfit_str = ", ".join([get_item_description(id) for id in best_combo_ids])
    best_outfit_info = {"description": outfit_str, "ids": best_combo_ids}
    explanation = f"긍정적: {req.temperature}°C의 {req.condition} 날씨에 진행되는 {req.event}에 가장 잘 어울리는 스타일입니다."
    end_time = time.time()
    return {"best_combination": best_outfit_info, "explanation": explanation, "best_score": best_score, "processing_time": end_time - start_time}

Overwriting serve_final.py


In [5]:
# --- 5. 서버 실행 및 ngrok 터널 생성 ---
import threading
import time
import requests
from pyngrok import ngrok
import uvicorn

def run_app():
  uvicorn.run("serve_final:app", host="0.0.0.0", port=8000, log_level="info")

thread = threading.Thread(target=run_app)
thread.start()
print("\n✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...")

print("⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다...")
start_time = time.time()
server_ready = False
while time.time() - start_time < 600:
    try:
        response = requests.get("http://localhost:8000/", timeout=5)
        if response.status_code == 404:
            print("✅ 서버가 응답하기 시작했습니다!")
            server_ready = True
            break
    except requests.exceptions.ConnectionError: time.sleep(5); print("...")
if not server_ready:
    print("❌ 10분이 지나도 서버가 준비되지 않았습니다. 런타임을 재시작하고 다시 시도해주세요.")
else:
    NGROK_TOKEN = "31HHkLEWGt90qDRoAWd6BqiGL9K_5fJsXN7LgijbtcAVwkwAS" # 🚨
    ngrok.set_auth_token(NGROK_TOKEN)
    ngrok.kill()
    public_url = ngrok.connect(8000)
    print(f"\n🎉 서버가 성공적으로 생성되었습니다. API 주소: {public_url}")
    print("이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.")

thread.join()


✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...
⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다...
...
...
...


INFO:     Started server process [24222]
INFO:     Waiting for application startup.


...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...
...
...
...


INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


...
...
INFO:     127.0.0.1:34878 - "GET / HTTP/1.1" 404 Not Found
✅ 서버가 응답하기 시작했습니다!

🎉 서버가 성공적으로 생성되었습니다. API 주소: NgrokTunnel: "https://39f17048bb40.ngrok-free.app" -> "http://localhost:8000"
이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.


INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     39.113.75.42:0 - "POST /recommend_outfit HTTP/1.1" 200 OK


KeyboardInterrupt: 